In [1]:
import pandas as pd
import requests as rq

In [3]:
# Definindo parâmetros da API
url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados'
params = {'dataInicial': '01/01/2025','dataFinal': '19/08/2025'}

res = rq.get(url, params=params, timeout=30)
res.raise_for_status()
dados = res.json()


df = pd.DataFrame(dados)


# Transforma o campo data pra padrão americano yyyy-mm-dd
df["data"] = pd.to_datetime(df["data"], dayfirst=True) # parâmetro dayfirst=True diz que o formato da data está no padrão brasileiro (dd/mm/yyyy)

# Transformando o valor para para tipo float
df["taxa_diaria_percentual"] = df["valor"].astype(float)

# Realizando a multiplicação de 252 (dias de negociação) para dados relacionados ao mercado conforme a fonte https://www.investopedia.com/terms/a/annualize.asp
# Nesse primeiro comando estou dividindo por 100 pra realizar o calculo de exponenciações
df["taxa_diaria_decimal"] = df["taxa_diaria_percentual"] / 100.0

# Anualização por 252 dias úteis conforme a documentação da fonte
df["taxa_selic"] = ((1 + df["taxa_diaria_decimal"]) ** 252 - 1) * 100

# Selecionando somente as colunas data e taxa_selic
df = df[['data', 'taxa_selic']]
df




,data,taxa_selic
0,2025-01-02,12.149954
1,2025-01-03,12.149954
2,2025-01-06,12.149954
3,2025-01-07,12.149954
4,2025-01-08,12.149954
...,...,...
152,2025-08-12,14.899981
153,2025-08-13,14.899981
154,2025-08-14,14.899981
155,2025-08-15,14.899981
